<a href="https://colab.research.google.com/github/jiw3026/SML/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
#데이터 불러오기
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')

In [ ]:
#결측치 0으로 처리
train.fillna(0,inplace = True)
test.fillna(0,inplace = True)

In [ ]:
#이상치 처리(IQR 25%~75%)
colL  = ['hour','hour_bef_temperature', 'hour_bef_precipitation', 'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility', 'hour_bef_visibility',
      'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'count']

train_iqr = train.copy()
for col in colL:
  Q1 = train_iqr[col].quantile(.25)
  Q3 = train_iqr[col].quantile(.75)
  IQR = Q3 - Q1
  train_del = train_iqr[((Q1-1.5*IQR) > train_iqr[col]) | ((Q3+1.5*IQR) < train_iqr[col])]
  train_iqr = train_iqr.drop(train_del.index, axis=0)
train_iqr

In [ ]:
#train 데이터 내에서 X_train, X_test, y_train, y_test 데이터셋 분리함.
train_x = train_iqr.drop(['count'],axis = 1)
train_y = train_iqr['count']
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

In [ ]:
#X_train 데이터 표준화
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

In [ ]:
#X_test 데이터 표준화
X_test=scaler.transform(X_test)

X_train : train에 사용하는 데이터의 x값

 y_train : train에 사용하는 데이터의 y값

 X_test : test(predict)에 사용할 데이터의 x값
 
 
  y_test: test(predict)에 사용할 데이터의 y값

  여기있는 X_train, X_test, y_train, y_test을 각자 모델 변수에 맞추어 사용하면 됩니다~~

----------------------------------------------------------------------------------------------------------------------------------------------------------------

<H1>그리드서치로 최적의 파라미터 찾기</H1>

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    '파라미터 1':[100,200],
    '파라미터 2': [6,8,10,12],
    '파라미터 3':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
}
grid_search = GridSearchCV(각자 모델 이름(), param_grid, cv="필요한 값", scoring='neg_mean_squared_error')
#예시: grid_search = GridSearchCV(XGBRegressor(random_state=42), param_grid, cv=2, n_jobs=1, scoring='neg_mean_squared_error')
#scoring='neg_mean_squared_error'을 꼭 넣어야 rmse에서 이상한 값이 안나옵니다.
grid_search.fit(X_train,y_train)

<H1>그리드서치에서 찾은 파라미터를 모델에 적용해주기</H1>

In [ ]:
모델 변수 = 각자 모델 이름(파라미터 1=200, 파라미터 2=0.8, 파라미터 3=0.7)
#예시: xgb_reg = XGBRegressor(objective='reg:squarederror', n_estimators=200, colsample_bylevel=0.8, colsample_bytree=0.7, eta=0.1, max_depth=6)
모델 변수.fit(X_train,y_train)

<H1>위에서 만든 최적의 파라미터를 적용한 모델로 rmse 구하기</H1>

In [ ]:
pred = 모델 변수.predict(X_test)
#예시: pred_xgb = xgb_reg.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print(rmse)

<H1>그리드서치 돌린다음 각 파라미터 값 별 rmse 구하기</H1>
<H3>모델 설명 쓸때 필요하신분 사용하세요.</H3>

In [ ]:
#grid_search는 grid_search = GridSearchCV(XGBRegressor(random_state=42, objective='reg:squar....이런식으로 그리드 서치 돌린거 적용한 모델 이름입니다. 
#각자 모델 변수에 맞춰서 아래에서 grid_search 이름을 다른걸로 바꿔서 돌리면 됩니다.
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df

In [ ]:
scores_df['rmse'] = np.sqrt(-scores_df['mean_test_score'])

In [ ]:
#각 파라미터 값 별 rmse 비교
#아래에 있는 'param_colsample_bylevel','param_colsample_bytree','param_eta','param_max_depth','param_n_estimators','mean_test_score'값들은 
#각자 모델에서 사용한 파라미터 변수들 이름입니다. 
#위의 scores_df 데이터프레임에서 보고 파라미터 변수들 이름 알아내서 아래 값들만 바꿔주면 됩니다.
scores_df[['param_colsample_bylevel','param_colsample_bytree','param_eta','param_max_depth','param_n_estimators','mean_test_score','rmse']].sort_values(
    'mean_test_score', ascending=False)